In [1]:
# add csv file
import pandas as pd
pd.set_option('display.max_columns', None)
url = 'https://raw.githubusercontent.com/rackhamsauer/Project_4/main/Data/clean_data.csv'
df = pd.read_csv(url)
df.head()




,id,screen_name,default_profile,description,created_at,geo_enabled,lang,statuses_count,followers_count,favourites_count,verified,account_type
0,16282004,eckates,1,My name is Enrique! ! I'm a highly experienced...,2008-09-14 13:20:09,0,en,11405,819,0,0,1
1,16740486,genebailey,0,"Author, Speaker, Father, Friend, Motivator, Re...",2008-10-14 18:11:24,1,en,520,219,36,0,1
2,17132768,patrickghoward,0,Patrick G Howard is an experienced project & p...,2008-11-03 16:51:00,0,en,4671,38877,6,0,1
3,18013384,DoggieCakes,0,Dog Bakery and Boutique - Retail (Events and O...,2008-12-10 06:44:57,1,en,8512,2069,70,0,1
4,21331733,DavidVarrone,0,Home Based Business and Personal Development C...,2009-02-19 20:46:02,0,en,245,6656,1,0,1


In [2]:
# Split the description column into individual words
from collections import Counter
import re

words_list = df['description'].str.split()
df['cleaned_description'] = df['description'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))
words_series = df['cleaned_description'].str.split(expand=True).stack()

word_counts = Counter(words_series)
words_dict = dict(word_counts)

len(list(words_dict))

62199

In [3]:
words_df = pd.DataFrame(list(words_dict.items()), columns=['Word', 'Count'])
words_df.head()

,Word,Count
0,my,3064
1,name,125
2,is,2204
3,enrique,3
4,im,1204


In [4]:
df['cleaned_description']

0        my name is enrique  im a highly experienced en...
1        author speaker father friend motivator restore...
2        patrick g howard is an experienced project  pr...
3        dog bakery and boutique  retail events and onl...
4        home based business and personal development c...
                               ...                        
29939    role stock taker past roles nanny sales assist...
29940                       kingdom landlord freecornbread
29941    bienvenids al twitter oficial de sergio dalma ...
29942    just a good guy wrapped up in a bad system\nac...
29943                                                     
Name: cleaned_description, Length: 29944, dtype: object

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import re

# Adjust the preprocessing function to be less restrictive
def preprocess(text):
    text = re.sub(r'\d+', '', text)  # Remove numbers
    return text

# Initialize CountVectorizer with the adjusted preprocessor
vectorizer = CountVectorizer(stop_words=None, max_features=1500, preprocessor=preprocess)

# Fit and transform the data again with the adjusted settings
X_adjusted = vectorizer.fit_transform(df['cleaned_description'])

non_zero_counts = X_adjusted.getnnz(1)
non_zero_counts

array([18,  6, 12, ...,  4, 11,  0])

In [6]:
# Create a DataFrame from the output
word_count_df = pd.DataFrame(X_adjusted.toarray(), columns=vectorizer.get_feature_names_out())

In [7]:
# Concatenate the original DataFrame with the word count DataFrame
description_df = pd.concat([df, word_count_df], axis=1)

In [8]:
description_df.to_csv('Data/description.csv', index=False)